Full code to make all waypoints from road sections 

In [1]:
#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
import glob
import os
try:
    sys.path.append(glob.glob('./carla/dist/carla-0.9.14-py3.7-win-amd64.egg')[0])
except IndexError:
    print('Couldn\'t import Carla egg properly')
import carla

try:
    sys.path.insert(0,r'C:\Users\ashokkumar\source\repos\AD\carla')
except IndexError:
    pass

from carla import ColorConverter as cc
from agents.navigation.behavior_agent import BehaviorAgent  
from agents.navigation.basic_agent import BasicAgent  

# connect to the sim 
client = carla.Client('localhost', 2000)

world = client.get_world()
world = client.load_world('Town02')

spectator = world.get_spectator()

# get map look at the map
town_map = world.get_map()
roads = town_map.get_topology()

# set up route generations

In [2]:
'''
this is a better option:
1. it allows to set a distance between waypoints
2. it gets them all without gaps
3. Do not need ti use route planner

#this takes about 4 minutes - in RL context, this only needs to run at the start or when you change a towns
'''
all_waypoints = town_map.generate_waypoints(1) #note distance between waypoints in meters

# make unique
unique_waypoints = []
for wp in all_waypoints:
    if len(unique_waypoints)==0:
        unique_waypoints.append(wp) #first waypoint is added regardless to start the list
    else:
        found = False
        for uwp in unique_waypoints: #check for same located waypoints and ignore if found
            if abs(uwp.transform.location.x - wp.transform.location.x) < 0.1 \
                            and abs(uwp.transform.location.y - wp.transform.location.y)<0.1 \
                            and abs(uwp.transform.rotation.yaw - wp.transform.rotation.yaw)<20:  #this checks same direction
                found = True
                break
        if not found:
            unique_waypoints.append(wp)

# draw all point in the sim for 60 seconds
for wp in unique_waypoints:
    world.debug.draw_string(wp.transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=60.0,
        persistent_lines=True)

#move spectator for top down view to see all points 
spectator_pos = carla.Transform(carla.Location(x=0,y=30,z=200), carla.Rotation(pitch=-90, yaw=0, roll=0))
spectator.set_transform(spectator_pos)

: 

In [68]:
'''
now we will see how this will be used in real examples, like during RL training
e.g. see how long it takes to find closest point to the car

'''
#remove any cars/clean up the sim
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
blueprint_library = world.get_blueprint_library()

# Now let's filter all the blueprints of type 'vehicle' and choose one
bp = blueprint_library.find('vehicle.tesla.model3')
transform = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(bp, transform)  
spectator = world.get_spectator()
spectator_transform =  vehicle.get_transform()
spectator_transform.location += carla.Location(z = 10.0)
world.get_spectator().set_transform(spectator_transform)


agent = BasicAgent(vehicle)
waypoint_counter = 0


In [77]:
my_waypoint = vehicle.get_transform().location
curr_distance = 1000
for wp in unique_waypoints:
    dist = my_waypoint.distance(wp.transform.location)
    if dist < curr_distance:
        curr_distance =  dist
        selected_wp = wp

next_wp = selected_wp.next(5.0)[0]
destination = next_wp.transform.location
agent.set_destination(destination)

In [78]:
while True:
    if agent.done():
        next_wp = selected_wp.next(5.0)[0]
        destination = next_wp.transform.location
        agent.set_destination(destination)
        world.debug.draw_string(next_wp.transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=255, b=0), life_time=2.0,
        persistent_lines=True)
        print("The target has been reached, setting next waypoint")
        selected_wp = next_wp
        waypoint_counter += 1

    if waypoint_counter > 100:
        print("We have reached the end of the waypoints")
        break
    vehicle.apply_control(agent.run_step())

We have reached the end of the waypoints


In [75]:
# example how to use this for measuring where the car is in relation to where it should be

vehicle_transform = vehicle.get_transform()
distance_to_wp = selected_wp.transform.location.distance(vehicle_transform.location)
direction_difference = (vehicle_transform.rotation.yaw - selected_wp.transform.rotation.yaw) % 180
print('deviation from waypoint:',distance_to_wp,'metres')
print('angle discrepancy:',direction_difference,'degrees')


deviation from waypoint: 325.9584045410156 metres
angle discrepancy: 89.96080780029297 degrees


In [76]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()


In [82]:
spectator_transform =  vehicle.get_transform()
world.get_spectator().set_transform(spectator_transform)

: 

In [81]:
for actor in world.get_actors().filter('*vehicle*'):
    print(actor)